In [3]:
import urllib
from bs4 import BeautifulSoup
import yaml

mainUrl = 'https://duosmium.org/results/'

# parse mainpage
fh = urllib.request.urlopen(mainUrl)
bs = BeautifulSoup(fh,'html.parser')

# obtain tournament urls
tourns = bs.find_all('a',{'class':'btn btn-outline-primary full-results'})
refs = ['https://duosmium.org'+i['href'] for i in tourns]
refs2 = [i.replace('/results/','/data/')+'.yaml' for i in refs]

In [2]:
import sqlite3

conn = sqlite3.connect('unosmium.sqlite',timeout=10)
cur = conn.cursor()

cur.executescript('''
    drop table if exists events;
    drop table if exists teams;
    drop table if exists places;
    drop table if exists tournaments;
    drop table if exists events_tournaments''')

In [ ]:
cur.executescript('''
    create table if not exists events(
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        event TEXT UNIQUE);
    create table if not exists teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        school TEXT,
        rank INTEGER);
    create table if not exists places(
        place INTEGER, 
        team_id INTEGER, 
        tournament_id INTEGER, 
        event_id INTEGER, 
        PRIMARY KEY (team_id,tournament_id,event_id));
    create table if not exists tournaments(
        id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
        tournament TEXT UNIQUE,
        division INTEGER,
        day DATE,
        location TEXT);
    create table if not exists events_tournaments(
        tournament_id INTEGER, 
        event_id INTEGER, 
        official INTEGER,
        PRIMARY KEY (tournament_id,event_id))''')

In [ ]:
ytxt = yaml.load((urllib.request.urlopen(refs2[3])).read().decode(),Loader=yaml.FullLoader)

In [ ]:
for ref in refs2:
    fh = urllib.request.urlopen(ref)
    txt = fh.read().decode()
    
    ytxt = yaml.load(txt,Loader=yaml.FullLoader)
    

In [ ]:
# tournaments
cur.execute('''
    INSERT OR IGNORE INTO tournaments(tournament,division,day,location)
    VALUES (?,?,?,?)''',
    (ytxt['Tournament'].get('name',None),
    1 if ytxt['Tournament'].get('division',None)=='C' else 0,
    ytxt['Tournament'].get('date',ytxt['Tournament'].get('start date',None)),
    ytxt['Tournament'].get('location',None)))
cur.execute('SELECT id FROM tournaments WHERE tournament=?',(ytxt['Tournament']['name'],))
tournament_id = cur.fetchone()[0]

In [ ]:
# create teams
from collections import Counter
schoolCounts = Counter([i['school'] for i in ytxt['Teams']])
schools = list()

for school,count in schoolCounts.items():
    if count<1: 
        print('Error: Invalid school counts:',ytxt['Tournament']['name'],'-',school)
        continue
    for i in range(1,count+1):
        schools.append((school,i))

# check valid team names and numbers
if len(schools) != len(ytxt['Teams']):
    print('Error: Invalid number of teams:',ytxt['Tournament']['name'])
    #continue
if len(schools) != len(set(schools)):
    print('Error: Duplicated team names:',ytxt['Tournament']['name'])
    #continue

# SQL stuff
cur.executemany('''INSERT OR IGNORE INTO teams(school,rank) VALUES (?,?)''',schools)

In [ ]:
# create team number - school,rank dictionary; rank ordered by team number for each school
teams = list(set([i['school'] for i in ytxt['Teams']]))
teamsCounter = {i:0 for i in teams}

teamNumbers = [[i['number'],i['school'],None] for i in ytxt['Teams']]
teamNumbers.sort()

for i in range(len(teamNumbers)):
    teamsCounter[teamNumbers[i][1]] += 1
    count = teamsCounter[teamNumbers[i][1]]
    teamNumbers[i][2] = count

teamNumbers2 = {i[0]:[i[1],i[2]] for i in teamNumbers}
nTeams = len(teamNumbers2)

In [ ]:
# get events data
events = [i['name'] for i in ytxt['Events']] # all events
trialEvents = [i['name'] for i in ytxt['Events'] if i.get('trial',False) or i.get('trialed',False)] # all trial/trialed events
# check valid events and trial events
if len(events)!=len(set(events)):
    print('Error: Duplicated events:',ytxt['Tournament']['name'])
    #continue
if len([i for i in ytxt['Events'] if len(i)>1])!=len(trialEvents):
    print('Warning: Unrecognized event tag:',ytxt['Tournament']['name'])
    
# SQL stuff
for event in events:
    cur.execute('''INSERT OR IGNORE INTO events(event) VALUES(?)''',(event,))
    cur.execute('SELECT id FROM events WHERE event=?',(event,))
    event_id = cur.fetchone()[0]
    cur.execute('''INSERT OR IGNORE INTO events_tournaments(tournament_id, event_id, official)
        VALUES (?,?,?)''',(tournament_id, event_id, 0 if event in trialEvents else 1))

In [ ]:
# places
for place in ytxt['Placings']:
    # locate team id
    school = teamNumbers2[place.get('team')]
    cur.execute('''SELECT id FROM teams WHERE school=? and rank=?''',(school[0],school[1]))
    team_id = cur.fetchone()[0]
    
    # event id
    cur.execute('SELECT id FROM events WHERE event=?',(place['event'],))
    event_id = cur.fetchone()[0]
    
    # main
    cur.execute('''INSERT OR IGNORE INTO places(place,team_id,tournament_id,event_id) VALUES (?,?,?,?)''',
        (place.get('place',nTeams+1), team_id, tournament_id, event_id))

conn.commit()

In [38]:
cur.close()